# Controlling the acceleration of the slashing strategy

Previously, we used an affine function to gauge the impact of our slashing on actors based on their individual failure rates. However, it might be more effective to implement a quadratic or even exponential function. This would rapidly eliminate actors who repeatedly fail, and allow those with fewer errors more time to adjust their strategies and become profitable.

For that purpose we need to find out which slash to apply based on wether it is a repeated offense or not.  
Intuitively, I think that the state of the user slash is a good indication of wether he has been acting bad lately or not. If a lot of money has already been slashed it means the user keeps behaving in a non desirable way. We should slash him harder to deter him of continuing.